# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
import types
from functools import wraps
import typer
from typing_extensions import Annotated
from fastcore.docments import *
from fastcore.all import *
from nbdev import cli
from nbdev import clean as n_clean
from rich import print

In [ ]:
app = typer.Typer()

In [ ]:


commands = {
    'clean':n_clean.nbdev_clean
}

for fname,func in commands.items():
    # Add to typer.app
    func = app.command()(func)    

    # Prep the annotations
    arguments = docments(func, full=True)
    for arg, meta in arguments.items():
        if (meta['anno'] is bool_arg): 
            meta['anno'] = bool
        func.__annotations__[arg] = Annotated[meta['anno'], typer.Argument()]    

    globals()[fname] = func


In [ ]:
clean

<function nbdev.clean.nbdev_clean(fname: Annotated[str, <typer.models.ArgumentInfo object>] = None, clear_all: Annotated[bool, <typer.models.ArgumentInfo object>] = False, disp: Annotated[bool, <typer.models.ArgumentInfo object>] = False, stdin: Annotated[bool, <typer.models.ArgumentInfo object>] = False) -> Annotated[inspect._empty, <typer.models.ArgumentInfo object>]>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()